### Charting

The chart library is based on [ChartJs](https://www.chartjs.org/). The MeshWeaver classes are wrappers around the javascript data structures defined in ChartJs and provide a fluent API to interface seemingless with C#. 

In [1]:
#r "nuget:MeshWeaver.Connection.Notebook, 2.0.0-preview1"

Installed Packages MeshWeaver.Connection.Notebook, 2.0.0-preview1

Loading extension script from `C:\Users\rolan\.nuget\packages\meshweaver.connection.notebook\2.0.0-preview1\interactive-extensions\dotnet\extension.dib`

Usage:
#!connect mesh --url https://<mesh-weaver-host>/kernel --kernel-name <kernel-name>


In [2]:
#!connect mesh https://localhost:65260/kernel --kernel-name mesh

Kernel added: #!mesh

### Including Libraries
Whereas for nuget packages we have to type

```csharp
    #r "nuget: MathNet.Numerics"
```

We can also include libraries which are already deployed with Mesh Weaver. Here, we include the Charting library, which we have deployed into the portal. This is why we omit the specifier nuget and just write

```csharp
    #r "MeshWeaver.Charting"
```


In [3]:
#r "MeshWeaver.Charting"
using MeshWeaver.Layout;
using MeshWeaver.Mesh;
using static MeshWeaver.Layout.Controls;
using Microsoft.DotNet.Interactive.Formatting;
using MeshWeaver.Charting;

### Pareto Distribution

The probability density function (PDF) of the Pareto distribution:

$$f(x) = \frac{\alpha x_0^\alpha}{x^{\alpha+1}}$$

where:

$\alpha$ is the shape parameter (tail index)
$x_m$ is the scale parameter (minimum value)
Pareto CDF Formula
The cumulative distribution function (CDF) of the Pareto distribution:

$$F(x) = 1 - (\frac{x}{x_0})^{-\alpha}$$

where:

$\alpha$ is the shape parameter (tail index)
$x_0$ is the scale parameter (minimum value)


In [4]:
double ParetoQuantile(double p, double alpha, double x0)
=> x0 * Math.Pow(1 - p, -1 / alpha);

In [5]:
var random = new Random();
var alpha = 1.5;
var x0 = 1.0;
var nSamples = 1000;
var samples = Enumerable.Range(0, nSamples).Select(i => ParetoQuantile(random.NextDouble(), alpha, x0)).OrderBy(x => x).ToArray();

var min = samples.First();
var max = samples.Last();
var nHistogram = 100;
var binWidth = (max - min) / nHistogram;
var histogram = samples.GroupBy(x => Math.Floor((x-min) / binWidth)).Select(g => (g.Key, Count:g.Count())).ToArray();


var barHistogram = DataSet.Bar(histogram.Select(x => x.Count / (double)nSamples), "PDF");
var pdfChart = Chart.Create(barHistogram)  
    .WithLabels(histogram.Select(x => (x.Key * binWidth + min).ToString("F1")))
    .ToControl()
    .WithStyle(style => style.WithWidth("700px").WithHeight("400px"));

pdfChart

In [6]:
var sum = 0.0;
var binWidthHalf = binWidth / 2;
var cdf = histogram.Select(x => (x.Key, Count: sum += x.Count)).ToArray();
var lineCdf = DataSet.Line(cdf.Select(x => x.Count / (double)nSamples), "CDF");

var cdfChart = Chart.Create(lineCdf)
    .WithLabels(cdf.Select(x => (x.Key * binWidth + min).ToString("F1")))
    .ToControl()
    .WithStyle(style => style.WithWidth("700px").WithHeight("400px"));

cdfChart

In [7]:
cdfChart = Chart.Create(lineCdf.WithStepped("before").WithTension(0.0).WithBorderWidth(2))
    .WithLabels(cdf.Select(x => (x.Key * binWidth + min).ToString("F1")))
    .ToControl()
    .WithStyle(style => style.WithWidth("700px").WithHeight("400px"));

cdfChart

In [8]:
Chart.Create(barHistogram, lineCdf)
    .WithLabels(cdf.Select(x => (x.Key * binWidth + min).ToString("F1")))
    .ToControl()
    .WithStyle(style => style.WithWidth("700px").WithHeight("400px"))

In [9]:
Stack
    .WithView(pdfChart)
    .WithView(cdfChart)
    .WithStyle(style => style.WithWidth("700px").WithHeight("800px"))